In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [2]:
# Load and preprocess the data
data = pd.read_csv('modified.csv')
data = data.iloc[:, 2:].dropna()
X, Y = data['transformed_text'], data['label_num']

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
def tokenize_function(texts):
    return tokenizer(
        texts.tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='pt'
    )

train_encodings = tokenize_function(X_train)
val_encodings = tokenize_function(X_test)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return item

train_dataset = CustomDataset(train_encodings, Y_train)
val_dataset = CustomDataset(val_encodings, Y_test)

In [12]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',  
)

In [13]:
# Define metrics computation
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    accuracy = accuracy_score(p.label_ids, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [14]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# Train the model 
trainer.train()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #because we were using google colab environement 
model.to(device)

In [ ]:
def preprocess_input(text, tokenizer, max_len, device):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )
    return {
        'input_ids': encoding['input_ids'].flatten().to(device),
        'attention_mask': encoding['attention_mask'].flatten().to(device),
    }


In [ ]:
def predict_label(text, model, tokenizer, max_len, device):
    inputs = preprocess_input(text, tokenizer, max_len, device)
    input_ids = inputs['input_ids'].unsqueeze(0)  # Add batch dimension
    attention_mask = inputs['attention_mask'].unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    return predicted_label

In [ ]:
# Test with user input
user_input = input("Enter a text: ")
max_len = 128
predicted_label = predict_label(user_input, model, tokenizer, max_len, device)

# Interpret the prediction
if predicted_label == 0:
    print("The model predicts that the input is Positive.")
else:
    print("The model predicts that the input is Negative.")